# Container App Authentication with Client Credentials Flow

1. Web App Configuration
-  On Container App - Set Easy Auth
-  Add /.default to allowed toekn audiences api://<client-id>/.default
- Save and Re-open add
  - Allow from Specifc Client Applcations
  - Alllow any Identity

2.  ON App Registration for Web App Client Id
- In "API Permissions" Grant ADmin Content
- On Expose an API - Add client application

3.  On Enterprise Application -> Permsisions "Grant Admin Content" 
- This will enable it to show up in next step

4.  ON App Registration that makes API call
- Add permission

In [99]:
import requests
import os
from dotenv import load_dotenv
import json
from pprint import pprint

# Azure Identity library for client credentials authentication
from azure.identity import ClientSecretCredential

## 2. Set API Configuration

Define the API endpoint URL for your Container App.

In [ ]:
# Load configuration from .env file if it exists
load_dotenv(override=True)

# API Configuration
# Replace this URL with your actual Container App URL
API_URL = os.getenv("API_URL", "webapp")
API_ENDPOINT = f"{API_URL}/hello"

print(f"API URL: {API_URL}")
print(f"API Endpoint: {API_ENDPOINT}")

In [ ]:
import os
def get_token_client_credentials():
    """
    Get a token using client credentials flow (no browser required).
    For this to work, you need to set CLIENT_SECRET in your .env file or environment variables.
    """
    # Load the client secret from environment variables
    caller_client_id = os.getenv("CALLER_CLIENT_ID") # Client app that's making the API call
    client_secret = os.getenv("CALLER_CLIENT_SECRET") # From .env

    app_id = os.getenv("APP_ID") # Target API app ID
    tenant_id = os.getenv("TENANT_ID") # From .env

    try:
        # Create credential with client secret
        credential = ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=caller_client_id,
            client_secret=client_secret
        )
        
        # Use the scope from Azure portal WITH /.default suffix (required for client credentials)
        scope = f"api://{app_id}/.default"
        print(f"Requesting token for scope: {scope}")
        
        token = credential.get_token(scope)
        print(f"✅ Successfully obtained access token using client credentials")
        return token.token, credential
    except Exception as e:
        print(f"❌ Error obtaining token: {str(e)}")
        return None, None

# Execute the client credentials flow
print("Trying client credentials flow...")
token, credential = get_token_client_credentials()

# Display token information
if token:
    print(f"Token obtained via client credentials (first 20 chars): {token[:20]}...")
else:
    print("Failed to obtain token via client credentials")

In [102]:
# Simple example of calling an API with just the standard OAuth2 header
def simple_api_call(endpoint, token):
    """
    Simple function to call an API with the standard OAuth2 Bearer token header
    """
    # Standard OAuth2 header - the most common approach
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    print(f"Calling API with standard OAuth2 header: {endpoint}")
    try:
        response = requests.get(endpoint, headers=headers)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ Success!")
            return response.json()
        else:
            print(f"❌ Failed with status code: {response.status_code}")
            print(response.text)
            return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

#You can uncomment this to try just the standard header approach
if token:
    result = simple_api_call(API_ENDPOINT, token)
    if result:
        print("API Response:")
        pprint(result)

Calling API with standard OAuth2 header: https://web-aca-app2.wittypebble-5a8d3c4d.eastus2.azurecontainerapps.io/hello
Status Code: 200
✅ Success!
API Response:
{'message': 'Hello, welcome to FastAPI!'}
Status Code: 200
✅ Success!
API Response:
{'message': 'Hello, welcome to FastAPI!'}
